In [ ]:
import torch
import os
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET

import matplotlib.pyplot as plt
from pathlib import Path

from torch.quasirandom import SobolEngine

from gpytorch.mlls import ExactMarginalLogLikelihood
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.kernels import MaternKernel, ScaleKernel
from gpytorch.constraints import Interval

from botorch import fit_gpytorch_mll
from botorch.utils.transforms import unnormalize, normalize
from botorch.models import SingleTaskGP
from botorch.models.transforms import Standardize
from botorch.optim import optimize_acqf
from botorch.acquisition import qLogExpectedImprovement
from botorch.sampling.stochastic_samplers import StochasticSampler

from helpers import (load_experiment_metadata, 
                    compute_nrmse_counts_all_edges, 
                    parse_loop_data_xml_to_pandas, 
                    create_taz_xml,
                    simulate_od,
                    od_xml_to_df,
                    xml2df_str)


In [ ]:
!Pwd

In [ ]:
# base_path = "/home/bench/Gitsrcs/origin_destination_bayes_opt"
# base_path = "/Users/osorio/HEC/Research/Group/FacultyCollaborations/SeongjinChoi_UMN/Code_BO/origin_destination_bayes_opt-main"
base_path = "/Users/chois/Gitsrcs/origin_destination_bayes_opt"
os.chdir(base_path)

In [ ]:
config_path = Path(base_path , 'config')
print(f"config_path: {config_path}")

config, sim_setup = load_experiment_metadata(config_path)

network_name = sim_setup['network_name']
model_name = sim_setup['model_name']

network_path = Path("network" , network_name)
taz2edge_xml = Path(base_path, network_path, 'taz.xml')
net_xml = Path(base_path, network_path, 'net.xml')
fixed_routes = Path(base_path, network_path, 'routes.csv')
file_gt_od = Path(base_path, network_path, 'od.xml')
additional_xml = Path(base_path, network_path, 'additional.xml')

out_path = f"output/{network_name}_{model_name}" 
Path(out_path).mkdir(parents=True, exist_ok=True)

gt_version_str = network_name           ## TODO : need to check if this is correct

EDGE_OUT_STR = f'edge_data_{network_name}.xml'
# suffix of simulation output edge file
TRIPS2ODS_OUT_STR = 'trips.xml'
SUMO_PATH = config["SUMO"]

sim_start_time = sim_setup['sim_start_time']
sim_end_time = sim_setup['sim_end_time']
sim_stat_freq_sec = sim_setup['sim_stat_freq_sec']
od_duration_sec = sim_setup['od_duration_sec']

n_init_search = sim_setup['n_init_search']

NITER = sim_setup["BO_niter"]
BATCH_SIZE = sim_setup["BO_batch_size"]
NUM_RESTARTS = sim_setup["BO_num_restarts"]
RAW_SAMPLES = sim_setup["BO_raw_samples"] 

simulation_run_path =f'{out_path}'

In [ ]:

# # taz2edge_xml = 'taz_new.xml'
# # net_xml = 'SFO.net.xml'
# # fixed_routes_xml = f'{base_path}/5hr_route_choice_set.csv'
# # od_duration_seconds = 5*60 

# # # duration of sample time for simulation output statistics
# # simulation_stat_freq_sec = od_duration_seconds
# # sim_end_time = od_duration_seconds
# # additional_xml = f'additional.add_statfreq{od_duration_seconds}.xml'

# # # suffix of simulation output edge file
# # EDGE_OUT_STR = 'edge_data_SFO.xml'
# # TRIPS2ODS_OUT_STR = 'trips.xml'
# # SUMO_PATH = '/usr/local/opt/sumo/share/sumo'

# od_duration_seconds = 30*60 

# # duration of sample time for simulation output statistics
# simulation_stat_freq_sec = od_duration_seconds
# sim_end_time = od_duration_seconds

# # TODO: it might be cleaner to replace this with a config file, i attached to my email an example. and one can define one config file per network. 
# network_name = "quickstart"
# model_name = "bo_vanilla"

# network_path = f"network/{network_name}"
# taz2edge_xml = f"{base_path}/{network_path}/taz.xml"
# net_xml = f"{base_path}/{network_path}/net.xml"
# fixed_routes = f"{base_path}/{network_path}/routes.csv"
# # od_xml = f"{network_path}/od.xml"       ## TODO : need to check if this is correct
# file_gt_od = f"{base_path}/{network_path}/od.xml"      ## TODO : need to check if this is correct
# # file_gt_edges                         ## TODO : need to check if this is necessary (not being used below)
# additional_xml = f"{base_path}/{network_path}/additional.xml"
# out_path = f"output/{network_name}_{model_name}"
# out_path = f"output/{network_name}_{model_name}"       ## TODO : need to check if this is correct
# # prefix_output = f"{out_path}/out"     ## TODO : need to check if this is correct
# gt_version_str = network_name           ## TODO : need to check if this is correct

# EDGE_OUT_STR = f'edge_data_{network_name}.xml'
# # suffix of simulation output edge file
# TRIPS2ODS_OUT_STR = 'trips.xml'
# # TODO I changed this path for it to work for me.
# SUMO_PATH = '/opt/homebrew/opt/sumo/share/sumo'
# #SUMO_PATH = "/usr/share/sumo"

# Path(out_path).mkdir(parents=True, exist_ok=True)




In [ ]:
gt_version_str = 'v4'

# gt v4:
mean_od_val = 100
num_ods = 10

print('if you want to optimize them all (~86k) set num_ods as defined in commented line below')
#num_ods = routes_df.shape[0]

In [ ]:
# od_xml = f'gt_od_{gt_version_str}.xml'
# file_gt = f'{base_path}/gt_od_{gt_version_str}.xml'
# file_gt_edges = f'{base_path}/gt_edges_{gt_version_str}.csv'
# prefix_output_gt = f'gt_{gt_version_str}'

In [ ]:
# Get GT OD
print("Reading:",file_gt_od)
tree = ET.parse(file_gt_od)
root = tree.getroot()
gt_od_df =  xml2df_str(root, 'tazRelation')

gt_od_df.head()

In [ ]:
print("Reading:",fixed_routes)
routes_df = pd.read_csv(fixed_routes, index_col=0)

In [ ]:
gt_od_df = od_xml_to_df(file_gt_od)

In [ ]:
gt_od_df.columns

## Vanilla BO


### Declare parameter space


In [ ]:
# TODO: let's put all import  statements at the top of the notebook

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
dtype = torch.double

### Declare search space
# dimensionality of input space

dim_od = gt_od_df.shape[0]
print(dim_od)

#bounds = torch.tensor([
#    [ gt_od_df['count'].astype(float).min() - 2 for _ in range(dim_od)],
#    [ gt_od_df['count'].astype(float).max() + 2 for _ in range(dim_od)]
#], device=device, dtype=dtype) 

bounds = torch.tensor([
    [ 0 for _ in range(dim_od)],
    [ 2000 for _ in range(dim_od)]
], device=device, dtype=dtype) 


bounds



Run GT simulation


In [ ]:
simulation_gt_run_path =f'{out_path}/ground_truth'
Path(simulation_gt_run_path).mkdir(parents=True, exist_ok=True)
prefix_output_gt = f'{simulation_gt_run_path}/sim'

sim_edge_out_gt = f'{prefix_output_gt}_{EDGE_OUT_STR}'
new_od_xml = f'{simulation_gt_run_path}/od.xml'

base_od = gt_od_df.copy()
gt_od_vals = gt_od_df['count'].astype(float).to_numpy()
curr_od = gt_od_vals.copy()
base_od['count'] = curr_od
base_od = base_od.rename(columns={'fromTaz':'from', 'toTaz':'to'})        
create_taz_xml(new_od_xml, base_od, od_duration_sec, base_path)

print(base_od)

# Run simulation

simulate_od(new_od_xml, 
            prefix_output_gt, 
            base_path, 
            net_xml, 
            taz2edge_xml, 
            additional_xml,
            routes_df,
            sim_end_time,
            TRIPS2ODS_OUT_STR)



Read output of GT simulation


In [ ]:
df_edge_gt, _, _ = parse_loop_data_xml_to_pandas(base_path, sim_edge_out_gt, prefix_output_gt,SUMO_PATH)
# picking at edges as GT edges
num_gt_edges = df_edge_gt.shape[0]
print("Number of GT edges:",num_gt_edges)
gt_edge_data = df_edge_gt\
    .sort_values(by=['interval_nVehContrib'], ascending=False)\
    .iloc[:num_gt_edges]

# gt_edge_data.shape
print(sim_edge_out_gt)


In [ ]:
gt_edge_data

In [ ]:
# Sample according to Sobol
sobol = SobolEngine(dim_od, scramble=True)
x_0 = sobol.draw(n_init_search).to(dtype=dtype).to(device)
print(x_0.shape)


# map the normalized into the original parameter space
train_X0 = unnormalize(x_0, bounds)
print(train_X0.shape)
train_X0

In [ ]:

#num_epsilon_iter = 2
ods_epsilon = []
loss_all = []
batch_data_i = []

# Base OD which we will update their count entries
base_od = gt_od_df.copy()
gt_od_vals = gt_od_df['count'].astype(float).to_numpy()

for i , x in enumerate(train_X0.tolist()):
#for i , x in enumerate(
#      [[ 94.66438596,  91.97375804, 101.82277249, 112.44778006,
#            105.33019264,  92.62166575,  99.8673423 ,  93.71928772,
#            116.16658554,  94.79717515],
#      [ 97.4, 114.9, 104.1, 100. , 109.1, 106.7,  87.8, 101.1, 113.9,109.4]]):
      print(f"########### OD: {i} ###########")
      print(x)
      
      Path(f'{simulation_run_path}/initial_search').mkdir(parents=True, exist_ok=True)
      new_od_xml = f'{simulation_run_path}/initial_search/gt_od_{gt_version_str}_{i}.xml'
      prefix_output_init = f'{simulation_run_path}/initial_search/sobol_{i}'

      # Generate OD
      #curr_od = gt_od_vals.copy()
      curr_od = np.array(x)

      print(f'total expected GT demand: {np.sum(curr_od)}')

      ###
      # create OD xml file 
      ###
      base_od['count'] = curr_od
      # round to 1 decimal point
      base_od['count'] = [round(elem, 1) for elem in base_od['count']]     
      base_od = base_od.rename(columns={'fromTaz':'from', 'toTaz':'to'})        
      create_taz_xml(new_od_xml, base_od, od_duration_sec, base_path)
      ods_epsilon.append(curr_od)

      # simulate gt od
      simulate_od(new_od_xml, 
                  prefix_output_init, 
                  base_path, 
                  net_xml, 
                  taz2edge_xml, 
                  additional_xml, 
                  routes_df,
                  sim_end_time,
                  TRIPS2ODS_OUT_STR)

      ## Compute loss
      #prefix_output = f'initial_search/sobol_{i}'
      sim_edge_out = f'{base_path}/{prefix_output_init}_{EDGE_OUT_STR}'
      print(sim_edge_out)
      curr_loop_stats, _, _ = parse_loop_data_xml_to_pandas(base_path, sim_edge_out,prefix_output_init,SUMO_PATH)
      curr_loss = compute_nrmse_counts_all_edges(gt_edge_data, curr_loop_stats)

      loss_all.append(curr_loss)
      print(f"############## loss: {curr_loss} ##############")

      # Parse training data
      df_curr = pd.DataFrame(curr_od.reshape(1,dim_od),
                        columns = [f"x_{i+1}" for i in range(dim_od)])
      df_curr['loss'] = curr_loss
      batch_data_i.append(df_curr)



In [ ]:
df_initial_bo = pd.concat(batch_data_i)
df_initial_bo.head()

In [ ]:
simulation_run_path

In [ ]:
# Save initial dataset
df_initial_bo.to_csv(f"{simulation_run_path}/initial_search/data_set_ods_0_2000.csv",index=None)


#### Bayesian optimization helpers


In [ ]:



def initialize_gp_model(train_X,train_Y):
    
    dim = train_X.size(dim=1)

    likelihood = GaussianLikelihood(noise_constraint=Interval(1e-8, 1e-3))
    covar_module = ScaleKernel(  # Use the same lengthscale prior as in the TuRBO paper
        MaternKernel(
            nu=2.5, ard_num_dims=dim, lengthscale_constraint=Interval(0.005, 4.0)
        )
    )

    gp_model = SingleTaskGP(
        train_X, train_Y, 
        covar_module=covar_module, likelihood=likelihood, 
        outcome_transform=Standardize(m=1)
    )

    gp_mll = ExactMarginalLogLikelihood(gp_model.likelihood, gp_model)
    
    return gp_model, gp_mll

In [ ]:
### Acquisition Function: q-EI
# Acquisition function

sampler = StochasticSampler(sample_shape=torch.Size([128])) # TODO : sample shape from sim_setup?
#qEI = qExpectedImprovement(gp_model, best_f=max(train_Y), sampler=sampler)


In [ ]:
def optimize_acqf_and_get_observation(acq_func,bounds):
    """Optimizes the acquisition function, and returns a new candidate."""

    dim = acq_func.model.train_inputs[0].size(dim=1)

    # optimize
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=torch.tensor([[0.0] * dim, [1.0] * dim], device=device, dtype=dtype),
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        options={"batch_limit": 5, "maxiter": 200},
    )

    # observe new values 
    new_x = candidates.detach()
    
    return unnormalize(new_x, bounds)


In [ ]:
# df_0 = pd.read_csv(base_path + f"/initial_search/data_set_ods_0_2000.csv")
df_0 = pd.read_csv(f"{simulation_run_path}/initial_search/data_set_ods_0_2000.csv")

In [ ]:
#/Users/rodrse/Downloads/ForSergio_nov26_2023_final/bayesian_optimization/vanilla_bo
### Run loop
best_value = []

# Data frame of current training data
df_training = df_0
df_training["bo_iteration"] = 0

df_edge_stats = pd.DataFrame()

#num_epsilon_iter = 2
bayes_opt_method = "bayesian_optimization/vanilla_bo_experiment_2"
ods_epsilon = []
loss_all = []
batch_data_i = []

# Base OD which we will update their count entries
base_od = gt_od_df.copy()
gt_od_vals = gt_od_df['count'].astype(float).to_numpy()

for i in range(NITER):

    # new_od_xml = f'{bayes_opt_method}/gt_od_{gt_version_str}_{i}.xml'
    # prefix_output = f'{bayes_opt_method}/bayesOpt_{i}'

    new_od_xml = f'{simulation_run_path}/od.xml'
    prefix_output_bo = f'{simulation_run_path}/BO/bayesOpt_{i}'
    Path(f'{simulation_run_path}/BO').mkdir(parents=True, exist_ok=True)

    ########
    # Start BO step
    ########

    print(f"########### BO iteration={i+1} ###########")

    # Obtain sampling locations x
    train_X = torch.from_numpy(
        df_training[[col for col in df_training.columns if "x" in col]].values
    ).to(device=device, dtype=dtype)

    # Normalize
    train_X_norm = normalize(train_X,bounds)

    # Obtain reponse data
    train_Y = -torch.from_numpy(df_training[["loss"]].values) # Take negative

    # best value so far
    best_y = train_Y.max()
    best_value.append(best_y)
    print(f"##### best_value={best_y} #####")

    print(f"Generating new sampling location(s)....")
    # Declare model with newest data
    gp_model, gp_mll = initialize_gp_model(train_X_norm,train_Y)

    # Fit model
    fit_gpytorch_mll(gp_mll)

    # Construct acquistion function 
    sampler = StochasticSampler(sample_shape=torch.Size([128]))
    # qEI = qExpectedImprovement(gp_model, best_f=best_y, sampler=sampler)
    qEI = qLogExpectedImprovement(gp_model, best_f=best_y, sampler=sampler)

    # Maximize acquisition function to get next observation
    x_i = optimize_acqf_and_get_observation(acq_func=qEI,bounds=bounds)

    # map the normalized into the original parameter space
    #x_i = unnormalize(x_i, bounds)
    x_i = x_i.cpu().detach().numpy()

    ########
    # End BO step
    ########    


    # Sample simulator (inner loop across all sampling locations within a batch)
    # TODO: Parallelize
    batch_data_i = []
    for j in range(BATCH_SIZE):
        loss_all = []
        print(f"########### Sampling location={j+1} ###########")

        # Generate OD
        #curr_od = gt_od_vals.copy()
        curr_od = x_i[j]

        print(f'total expected GT demand: {np.sum(curr_od)}')

        base_od['count'] = curr_od
        # round to 1 decimal point
        base_od['count'] = [round(elem, 1) for elem in base_od['count']]     
        base_od = base_od.rename(columns={'fromTaz':'from', 'toTaz':'to'})        
        create_taz_xml(new_od_xml, base_od, od_duration_sec, base_path)

        # simulate gt od
        simulate_od(new_od_xml, 
                    prefix_output_bo, 
                    base_path, 
                    net_xml, 
                    taz2edge_xml, 
                    additional_xml, 
                    routes_df,
                    sim_end_time,
                    TRIPS2ODS_OUT_STR)

        ## Compute loss
        sim_edge_out = f'{base_path}/{prefix_output_bo}_{EDGE_OUT_STR}'
        print(sim_edge_out)
        curr_loop_stats, _, _ = parse_loop_data_xml_to_pandas(base_path, sim_edge_out,prefix_output_bo,SUMO_PATH)
        
        curr_loss = compute_nrmse_counts_all_edges(gt_edge_data, curr_loop_stats)
        loss_all.append(curr_loss)
        print(f"############## loss: {curr_loss} ##############")

        # Parse training data
        df_j = pd.DataFrame(x_i[j].reshape(1,dim_od),
                            columns = [f"x_{i+1}" for i in range(dim_od)])
        df_j['loss'] = curr_loss
        batch_data_i.append(df_j)

        curr_loop_stats['bo_iteration'] = i
        curr_loop_stats['batch'] = j
        df_edge_stats = pd.concat([df_edge_stats, curr_loop_stats])

    df_i = pd.concat(batch_data_i)
    df_i["bo_iteration"] = i+1

    df_training = pd.concat([df_training,df_i])


In [ ]:
df_training.shape

In [ ]:
# print(base_path + f"/{bayes_opt_method}/data_set_bayes_opt.csv")
print(f"{simulation_run_path}/BO/data_set_bayes_opt.csv")
# df_training.to_csv(base_path + f"/{bayes_opt_method}/data_set_bayes_opt.csv",index=None)
df_training.to_csv(f"{simulation_run_path}/BO/data_set_bayes_opt.csv",index=None)

print(f"{simulation_run_path}/BO/df_edge_stats.csv")
df_edge_stats.to_csv(f"{simulation_run_path}/BO/df_edge_stats.csv",index=None)

## plot trajectories


In [ ]:
df_training = pd.read_csv(f"{simulation_run_path}/BO/data_set_bayes_opt.csv")
df_edge_stats = pd.read_csv(f"{simulation_run_path}/BO/df_edge_stats.csv")

df_plot = df_training.query('bo_iteration>0')
x = df_plot['bo_iteration']
y = df_plot['loss'].cummin()

plt.plot(x, y)
#plt.legend(title='Parameter where:')
plt.xlabel('BO epochs')
plt.ylabel('Best NRMSE')
# plt.show()
plt.savefig(f"{simulation_run_path}/bo_nrmse.png")


In [ ]:
if df_edge_stats.batch.drop_duplicates().shape[0] > 1:
    raise('This needs updating once we start using batches')

losses = []
for o1 in range(NITER): #num_epsilon_iter):
    curr_edge_stats = df_edge_stats[df_edge_stats.bo_iteration == o1]
    df1b = gt_edge_data.merge(curr_edge_stats, on=['edge_id'], how='left', suffixes=('_gt', '_bo'))
    curr_loss = compute_nrmse_counts_all_edges(gt_edge_data, curr_edge_stats)
    losses.append(curr_loss)



In [ ]:
# disable interactive mode
plt.ioff()

if df_edge_stats.batch.drop_duplicates().shape[0] > 1:
    raise('This needs updating once we start using batches')

Path(f"{simulation_run_path}/figs").mkdir(parents=True, exist_ok=True)

losses = []
for o1 in range(NITER): #num_epsilon_iter):
    curr_edge_stats = df_edge_stats[df_edge_stats.bo_iteration == o1]
    df1b = gt_edge_data.merge(curr_edge_stats, on=['edge_id'], how='left', suffixes=('_gt', '_bo'))
    curr_loss = compute_nrmse_counts_all_edges(gt_edge_data, curr_edge_stats)
    losses.append(curr_loss)

    # find idx of min loss
    idx_min = np.argmin(losses)
    o1 = idx_min

    curr_edge_stats = df_edge_stats[df_edge_stats.bo_iteration == o1]
    df1b = gt_edge_data.merge(curr_edge_stats, on=['edge_id'], how='left', suffixes=('_gt', '_bo'))
    curr_loss = compute_nrmse_counts_all_edges(gt_edge_data, curr_edge_stats)

    plt.figure()    
    # plotting diagonal line that represents a perfect data fit
    max_val = np.max([df1b.interval_nVehContrib_gt.max(), df1b.interval_nVehContrib_bo.max()])
    vec = np.arange(max_val)
    plt.plot(vec, vec, 'r-')
    plt.plot(df1b.interval_nVehContrib_gt, df1b.interval_nVehContrib_bo, 'x') 
    # plt.title(f'BO epochs: {o1}; loss: {curr_loss}')
    plt.xlabel('GT edge counts') 
    plt.ylabel('Simulated edge counts') 
    plt.savefig(f"{simulation_run_path}/figs/{o1}_bo_edge_counts.png")


    # plot of fit to GT OD vs ET OD
    plt.figure()
    # get the OD values with the best loss
    curr_od = df_training.query('bo_iteration==@o1').iloc[0][[col for col in df_training.columns if "x" in col]].values
    # bar graph side by side by x axis
    width = 0.35
    plt.bar(np.arange(len(curr_od)), curr_od, width, label='BO')
    plt.bar(np.arange(len(gt_od_vals)) + width, gt_od_vals, width, label='GT')
    plt.legend()
    plt.xlabel('OD pair')
    plt.ylabel('Demand')
    # plt.title(f'BO iteration: {o1}')
    plt.savefig(f"{simulation_run_path}/figs/{o1}_bo_od.png")


In [ ]:
# draw a joint seaborn plot for initial (o1 = 0) and best (o1 = idx_min) iteration


import seaborn as sns
import matplotlib.pyplot as plt

o1 = 0
curr_edge_stats = df_edge_stats[df_edge_stats.bo_iteration == o1]
df1b = gt_edge_data.merge(curr_edge_stats, on=['edge_id'], how='left', suffixes=('_gt', '_bo'))
curr_loss = compute_nrmse_counts_all_edges(gt_edge_data, curr_edge_stats)

o2 = idx_min
curr_edge_stats = df_edge_stats[df_edge_stats.bo_iteration == o2]
df2b = gt_edge_data.merge(curr_edge_stats, on=['edge_id'], how='left', suffixes=('_gt', '_bo'))

# define another df to draw joint plot // hue = bo_iteration
df1b['bo_iteration'] = o1
df2b['bo_iteration'] = o2
df1b['type'] = 'initial'
df2b['type'] = 'best'

df_joint = pd.concat([df1b, df2b])

# plotting diagonal line that represents a perfect data fit
max_val = np.max([df1b.interval_nVehContrib_gt.max(), df1b.interval_nVehContrib_bo.max()])
vec = np.arange(max_val)

g = sns.jointplot(data=df_joint, x='interval_nVehContrib_gt', y='interval_nVehContrib_bo', hue='type')
# x label
g.ax_joint.set_xlabel('GT edge counts')
# y label
g.ax_joint.set_ylabel('Simulated edge counts')
g.ax_joint.plot(vec, vec, 'r-')

# save the plot
plt.savefig(f"{simulation_run_path}/{network_name}_jointplot_initial_best.png")
